# Understand Microsoft Agent Framework with code samples

----
Microsoft Agent Framework is the open-source engine for building production-ready agentic AI applications. It unifies the enterprise-ready foundations of Semantic Kernel with the innovative orchestration patterns of AutoGen, providing a single framework for both experimentation and production deployment.

| Components | Description |
|------------|-------------|
| **1. Chat Agents** | Core agents that handle conversational interactions using any compatible chat client (Azure OpenAI, OpenAI, Azure AI, etc.) |
| **2. Tools & Functions** | Extensible capabilities including built-in tools (Code Interpreter, File Search, Bing Grounding) and custom function tools |
| **3. Multi-Agent Orchestration** | Patterns for coordinating multiple agents (Sequential, Concurrent, Group Chat, Handoff, Magentic) |
| **4. Open Standards** | Native support for MCP (Model Context Protocol), A2A (Agent-to-Agent), and OpenAPI for interoperability |

![Microsoft Agent Framework](https://devblogs.microsoft.com/foundry/wp-content/uploads/sites/89/2025/09/AgentStack.png)

## Microsoft Agent Framework Architecture
---

Microsoft Agent Framework represents the evolution and unification of Semantic Kernel and AutoGen, bringing together the best of both worlds:

**Key Design Principles:**

1. **Open Standards First** - Built on MCP, A2A, and OpenAPI for maximum interoperability
2. **Research to Production Pipeline** - Cutting-edge orchestration patterns from Microsoft Research, production-ready
3. **Extensible by Design** - Modular architecture with pluggable components
4. **Enterprise Ready** - Built-in observability, security, durability, and human-in-the-loop support

**Comparison with Semantic Kernel:**
- Simplified agent creation without Kernel coupling
- Native thread management built into agents
- Inline tool registration without attribute decorators
- Direct integration with Azure AI Foundry Agent Service

**Comparison with AutoGen:**
- Unified message types (ChatMessage with clear roles)
- Graph-based Workflow API with checkpointing
- Built-in OpenTelemetry observability
- Stronger composability and durability for multi-agent systems

## Learn More:
- [Microsoft Agent Framework Documentation](https://learn.microsoft.com/en-us/agent-framework/)
- [GitHub Repository](https://github.com/microsoft/agent-framework)
- [Azure AI Foundry](https://ai.azure.com/)
- [Agent Framework Blog](https://devblogs.microsoft.com/foundry/introducing-microsoft-agent-framework-the-open-source-engine-for-agentic-ai-apps/)


This notebook will explore the core capabilities of Microsoft Agent Framework using Azure AI and Azure OpenAI as the backend services.

# Setup

Install the Microsoft Agent Framework packages. The framework is modular - you can install just what you need.

Import required libraries for the notebook.

In [1]:
import asyncio
import os
from typing import Annotated
from dotenv import load_dotenv

# Microsoft Agent Framework core
from agent_framework import ChatAgent, ChatMessage, TextContent, Role

# Azure integrations
from agent_framework.azure import AzureOpenAIChatClient, AzureAIAgentClient
from azure.identity.aio import DefaultAzureCredential, AzureCliCredential

# Tool decorators
from agent_framework import ai_function

load_dotenv(override=True)
print("✓ Libraries imported successfully")

✓ Libraries imported successfully


Configure environment variables. You can use either Azure OpenAI or Azure AI Foundry endpoints.

In [2]:
# Check required environment variables
print("Checking environment variables...")

# For Azure OpenAI
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_version = os.getenv("AZURE_OPENAI_API_VERSION")

# For Azure AI Foundry
azure_project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
azure_ai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

if azure_openai_endpoint and azure_openai_deployment:
    print("✓ Azure OpenAI configuration found")
    SERVICE_TYPE = "azure_openai"
else:
    print("❌ Missing required environment variables!")
    print("\nFor Azure OpenAI, set:")
    print("  - AZURE_OPENAI_ENDPOINT")
    print("  - AZURE_OPENAI_API_KEY")
    print("  - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
    print("\nOR for Azure AI Foundry, set:")
    print("  - AZURE_AI_PROJECT_ENDPOINT")
    print("  - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
    
print(f"\nUsing service: {SERVICE_TYPE}")
azure_openai_deployment

Checking environment variables...
✓ Azure OpenAI configuration found

Using service: azure_openai


'gpt-4.1-mini'

# Case 1: Basic Agent Creation and Execution

In Microsoft Agent Framework, creating an agent is straightforward. Unlike Semantic Kernel which requires a Kernel instance, Agent Framework agents are created directly from chat clients.

## 🧪 Case 1.1: Simple Agent with Azure OpenAI
---
Create a basic ChatAgent using AzureOpenAIChatClient. The agent handles thread management automatically.

In [3]:
# Create a chat client and agent with Azure OpenAI
azure_openai_chat_client = AzureOpenAIChatClient(
    deployment_name=azure_openai_deployment,
    endpoint=azure_openai_endpoint,
    api_key=azure_openai_key,
    api_version=azure_openai_version
)

# Create an agent - no Kernel needed!
agent = ChatAgent(
    chat_client=azure_openai_chat_client,
    instructions="You are a helpful assistant that provides concise, informative answers.",
    name="SimpleAssistant"
)

# Run the agent with a simple question
async def run_simple_agent():
    result = await agent.run("Tell me about Microsoft in 3 sentences.")
    print(f"Agent: {result.text}")

await run_simple_agent()

Agent: Microsoft is a multinational technology company founded in 1975 by Bill Gates and Paul Allen, known for developing the Windows operating system and Office productivity suite. It is a global leader in software, hardware, cloud computing, and artificial intelligence services. Microsoft also produces devices like the Xbox gaming console and Surface tablets, serving both consumer and enterprise markets worldwide.


## 🧪 Case 1.2: Streaming Response
---
Agent Framework supports streaming responses for real-time output.

In [4]:
async def run_streaming_agent():
    print("Agent: ", end="", flush=True)
    async for update in agent.run_stream("Explain what agentic AI is in simple terms."):
        if update.text:
            print(update.text, end="", flush=True)
    print()  # New line after streaming

await run_streaming_agent()

Agent: Agentic AI refers to artificial intelligence systems designed to act independently, make decisions, and take actions on their own to achieve specific goals—much like an agent. Instead of just following direct instructions, agentic AI can plan, adapt, and interact with its environment to accomplish tasks.


## 🧪 Case 1.3: Multi-turn Conversation with Thread
---
Agents automatically maintain conversation context using threads.

In [5]:
async def multi_turn_conversation():
    
    # Create a new thread.
    thread = agent.get_new_thread()
    
    response1 = await agent.run("My name is Alex and I love Python programming.", thread=thread)
    print(f"Agent: {response1.text}\n")
    
    # Get the thread from the result
    serialized = await thread.serialize()

    # Later, deserialize and continue conversation
    new_thread = await agent.deserialize_thread(serialized)
    
    # Second message - uses the same thread to maintain context
    response2 = await agent.run("What's my name?", thread=new_thread)
    print(f"Agent: {response2.text}\n")
    
    # Third message - agent remembers both previous exchanges
    response3 = await agent.run("What programming language did I say I love?", thread=new_thread)
    print(f"Agent: {response3.text}")

await multi_turn_conversation()

Agent: Nice to meet you, Alex! If you have any questions about Python programming or need help with a project, feel free to ask!

Agent: Your name is Alex.

Agent: You said you love Python programming.


# Case 2: Agents with Custom Tools/Functions
---
Agent Framework makes it easy to extend agents with custom functions using the `@ai_function` decorator. <br>
Define custom functions and register them with the agent. No plugin wrappers needed!

### 🧪 Case 2.1.1: Azure Open AI Client with Custom Function Tools

In [6]:
# Define custom functions using @ai_function decorator
@ai_function(description="Get the current weather for a location")
def get_weather(location: Annotated[str, "The city name"]) -> str:
    """Returns mock weather data for the given location."""
    # In a real scenario, this would call a weather API
    weather_data = {
        "Seoul": "Sunny, 22°C",
        "New York": "Cloudy, 18°C",
        "London": "Rainy, 15°C",
        "Tokyo": "Clear, 20°C"
    }
    return weather_data.get(location, f"Weather data not available for {location}")

@ai_function(description="Get the current time in a timezone")
def get_time(timezone: Annotated[str, "Timezone (e.g., 'UTC', 'EST', 'KST')"]) -> str:
    """Returns mock time for the given timezone."""
    from datetime import datetime
    # Simplified for demo - in production use proper timezone handling
    return f"Current time in {timezone}: {datetime.now().strftime('%H:%M:%S')}"

# Create an agent with these tools
weather_agent = ChatAgent(
    chat_client=azure_openai_chat_client,
    instructions="You are a helpful assistant that can provide weather and time information.",
    name="WeatherAssistant",
    tools=[get_weather, get_time]
)

# Test the agent with tool calls
async def test_tools():
    result = await weather_agent.run("What's the weather like in Seoul?")
    print(f"Agent: {result.text}\n")
    
    result2 = await weather_agent.run("What time is it in KST?")
    print(f"Agent: {result2.text}")

await test_tools()

Agent: The weather in Seoul is currently sunny with a temperature of 22°C.

Agent: The current time in Korea Standard Time (KST) is 01:15 AM.


### 🧪 Case 2.1.2: Azure AI Agent Client with Custom Function Tools

In [7]:
# Azure CLI 로그인은 터미널에서 먼저 수행해주세요:
# az login --use-device-code
# az account set --subscription "your-subscription-id"

# 현재 로그인 상태 및 subscription 확인
!az account show --output table

EnvironmentName    HomeTenantId                          IsDefault    Name                                     State    TenantDefaultDomain    TenantDisplayName         TenantId
-----------------  ------------------------------------  -----------  ---------------------------------------  -------  ---------------------  ------------------------  ------------------------------------
AzureCloud         16b3c013-d300-468d-ac64-7eda0820b6d3  True         MCAPS-Hybrid-REQ-86467-2024-hyokeunchoi  Enabled  fdpo.onmicrosoft.com   Microsoft Non-Production  16b3c013-d300-468d-ac64-7eda0820b6d3


In [8]:
# Menu class
class MenuTool:
    #Azure AI Agent doesn't need to use decorator
    #@ai_function(description="Get the specials from the menu")
    def get_specials(self) -> str:
        """Returns the daily specials."""
        return """
        Special Soup: Clam Chowder
        Special Salad: Cobb Salad
        Special Drink: Chai Tea
        """

    #@ai_function(description="Get the price of a menu item")
    def get_item_price(self, menu_item: Annotated[str, "The name of the menu item"]) -> str:
        """Returns the price of the requested item."""
        return "$9.99"

# Create agent using Azure AI
async def azure_ai_agent_demo():

    credential = AzureCliCredential()
    
    # Create an instance of the tool
    menu_tool = MenuTool()
    async with AzureAIAgentClient(
        async_credential=credential,
        project_endpoint=azure_project_endpoint,
        model_deployment_name=azure_ai_deployment
    ) as azure_ai_agent_client:
    
        ai_agent = ChatAgent(
            chat_client=azure_ai_agent_client,
            instructions="You are a helpful restaurant assistant. Answer questions about the menu.",
            name="MenuAssistant",
            tool_choice="auto",
            tools=[menu_tool.get_specials, menu_tool.get_item_price],
            # When conversation_id is set, store must be True for service-managed threads.
            store=True
        )
    
    # Test the agent
    questions = [
        "What are today's specials?",
        "How much does the Clam Chowder cost?"
    ]
    
    for question in questions:
        print(f"\nUser: {question}")
        result = await ai_agent.run(question)
        print(f"Agent: {result.text}")

await azure_ai_agent_demo()


User: What are today's specials?


Agent: Today's specials are:
- Special Soup: Clam Chowder
- Special Salad: Cobb Salad
- Special Drink: Chai Tea

Would you like to know the prices or details of any of these specials?

User: How much does the Clam Chowder cost?
Agent: The Clam Chowder costs $9.99.


### 🧪 Case 2.1.3: Azure OpenAI Responses Client with Custom Function Tools

In [11]:
from agent_framework.azure import AzureOpenAIResponsesClient
from azure.identity import AzureCliCredential

# Menu class
class MenuTool:
    def get_specials(self) -> str:
        """Returns the daily specials."""
        return """
        Special Soup: Clam Chowder
        Special Salad: Cobb Salad
        Special Drink: Chai Tea
        """

    def get_item_price(self, menu_item: Annotated[str, "The name of the menu item"]) -> str:
        """Returns the price of the requested item."""
        return "$9.99"

# Create agent using Azure OpenAI Responses Client
async def azure_openai_responses_demo():
    """
    Demonstrates AzureOpenAIResponsesClient with both non-streaming and streaming responses.
    This client provides structured response generation using Azure OpenAI.
    """
    
    # For authentication, run `az login` command in terminal
    menu_tool = MenuTool()
    
    # Create agent using AzureOpenAIResponsesClient
    agent = AzureOpenAIResponsesClient(
            endpoint=azure_openai_endpoint,
            api_key=azure_openai_key,
            deployment_name=azure_openai_deployment,
            api_version="preview" # Use "preview" for Azure OpenAI Responses. Currently, the api_version must be "preview".
            ).create_agent(
        instructions="You are a helpful restaurant assistant. Answer questions about the menu.",
        tools=[menu_tool.get_specials, menu_tool.get_item_price],
        
    )
    
    # Test questions
    questions = [
        "What are today's specials?",
        "How much does the Clam Chowder cost?"
    ]
    
    # 1. Non-streaming Example (get complete result at once)
    print("=== Non-streaming Response ===\n")
    for question in questions:
        print(f"User: {question}")
        result = await agent.run(question)
        print(f"Agent: {result.text}\n")
    
    print("-" * 80 + "\n")
    
    # 2. Streaming Example (get results as they are generated)
    print("=== Streaming Response ===\n")
    for question in questions:
        print(f"User: {question}")
        print("Agent: ", end="", flush=True)
        async for chunk in agent.run_stream(question):
            if chunk.text:
                print(chunk.text, end="", flush=True)
        print("\n")

await azure_openai_responses_demo()

=== Non-streaming Response ===

User: What are today's specials?


Agent: Today's specials are:
- Special Soup: Clam Chowder
- Special Salad: Cobb Salad
- Special Drink: Chai Tea

Would you like to know the prices or details of any of these specials?

User: How much does the Clam Chowder cost?
Agent: The Clam Chowder costs $9.99.

--------------------------------------------------------------------------------

=== Streaming Response ===

User: What are today's specials?
Agent: Today's specials are:
- Special Soup: Clam Chowder
- Special Salad: Cobb Salad
- Special Drink: Chai Tea

Would you like to know the prices or details of any of these?

User: How much does the Clam Chowder cost?
Agent: The Clam Chowder costs $9.99.



# Case 3: Understand the differences b/w Agent Clients

Take a look at the table below to understand the differences between three main agent clients in Microsoft Agent Framework: `AzureOpenAIChatClient`, `AzureAIAgentClient`, and `AzureOpenAIResponsesClient`.

| Case | Client Type | Tool Used | Execution Location |
|------|------------|-----------|-------------------|
| **3.1** | `AzureOpenAIChatClient` | Custom `@ai_function` | **Local (subprocess)** |
| **3.2** | `AzureAIAgentClient` | `HostedCodeInterpreterTool` | **Azure AI Service** |
| **3.3** | `AzureOpenAIResponsesClient` | `HostedCodeInterpreterTool` | **Azure OpenAI Service** | 

### When to Use Each Approach:

- **Use Case 3.1** when:
  - You need full control over code execution
  - You want to run code in your own environment
  - You don't have access to Azure AI services
  - You need custom execution logic (e.g., specific Python packages)

- **Use Case 3.2** when:
  - You're building enterprise applications
  - You need managed, secure code execution
  - You want to leverage Azure AI Foundry features
  - You need integration with other Azure AI services

- **Use Case 3.3** when:
  - You're using Azure OpenAI Responses API
  - You want to extract and inspect generated code
  - You need structured response format

## 🧪 Case 3.1: Azure Open AI Client with function calling (runs on subprocess)

In [12]:
import subprocess
import tempfile
import os

# Custom Python code execution function
@ai_function(description="Execute Python code and return the result")
def execute_python_code(code: Annotated[str, "Python code to execute"]) -> str:
    """
    Executes Python code in a temporary file and returns the output.
    
    Args:
        code: Python code to execute
        
    Returns:
        The output of the code execution or error message
    """
    try:
        # Create a temporary file to store the code
        with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as temp_file:
            temp_file.write(code)
            temp_file_path = temp_file.name
        
        # Execute the Python code
        result = subprocess.run(
            ['python', temp_file_path],
            capture_output=True,
            text=True,
            timeout=5  # 5 seconds timeout for safety
        )
        
        # Clean up
        os.unlink(temp_file_path)
        
        # Return output or error
        if result.returncode == 0:
            return f"Success:\n{result.stdout}" if result.stdout else "Code executed successfully with no output"
        else:
            return f"Error:\n{result.stderr}"
            
    except subprocess.TimeoutExpired:
        os.unlink(temp_file_path)
        return "Error: Code execution timed out (5 seconds limit)"
    except Exception as e:
        return f"Error: {str(e)}"
    
async def code_interpreter_demo_using_subprocess():
    
    thread = agent.get_new_thread()

    code_agent = ChatAgent(
        chat_client=azure_openai_chat_client,
        instructions="You are a helpful assistant that can write and get the result of the Python code.",
        name="CodeAssistant",
        tools=[execute_python_code]
    )
    
    # Ask the agent to solve a problem using code
    task = "Calculate the first 10 numbers in the Fibonacci sequence. Provide the result as a Python list."
    print(f"User: {task}")
    
    result = await code_agent.run(task, thread=thread)
    print(f"\nAgent: {result.text}")

await code_interpreter_demo_using_subprocess()

User: Calculate the first 10 numbers in the Fibonacci sequence. Provide the result as a Python list.



Agent: The first 10 numbers in the Fibonacci sequence are: [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]


## 🧪 Case 3.2: Azure AI Agent Client with Hosted Code Interpreter Tool (runs on server)

In [13]:

from agent_framework import HostedCodeInterpreterTool

async def code_interpreter_demo_using_subprocess():
    credential = AzureCliCredential()
    # Create agent with code interpreter capability
    code_agent = ChatAgent(
        chat_client=AzureAIAgentClient(
            async_credential=credential,
            project_endpoint=azure_project_endpoint,
            model_deployment_name=azure_ai_deployment
        ),
        instructions="You are a helpful assistant that can write and execute Python code.",
        name="CodeAssistant",
        tools=[HostedCodeInterpreterTool()]
    )
    
    # Ask the agent to solve a problem using code
    task = "Calculate the first 10 numbers in the Fibonacci sequence."
    print(f"User: {task}")
    
    result = await code_agent.run(task)
    print(f"\nAgent: {result.text}")

await code_interpreter_demo_using_subprocess()


User: Calculate the first 10 numbers in the Fibonacci sequence.



Agent: The first 10 numbers in the Fibonacci sequence are:

0, 1, 1, 2, 3, 5, 8, 13, 21, 34


## 🧪 Case 3.3: Azure OpenAI Responses Client with Hosted Code Interpreter Tool (runs on server)

In [14]:
from agent_framework import ChatAgent, ChatResponse, HostedCodeInterpreterTool
from agent_framework.azure import AzureOpenAIResponsesClient
from azure.identity import AzureCliCredential
from openai.types.responses.response import Response as OpenAIResponse
from openai.types.responses.response_code_interpreter_tool_call import ResponseCodeInterpreterToolCall

async def code_interpreter_responses_demo():
    """
    Demonstrates using HostedCodeInterpreterTool with Azure OpenAI Responses Client.
    
    This example shows how to execute Python code on Azure OpenAI service using
    the Responses API with the hosted code interpreter tool.
    """
    print("=== Azure OpenAI Responses Client with Hosted Code Interpreter ===\n")
    
    # For authentication, run `az login` command in terminal
    agent = ChatAgent(
        chat_client=AzureOpenAIResponsesClient(
            credential=AzureCliCredential(),
            endpoint=azure_openai_endpoint,
            deployment_name=azure_openai_deployment,
            api_version="preview" # Use "preview" for Azure OpenAI Responses. Currently, the api_version must be "preview".
        ),
        instructions="You are a helpful assistant that can write and execute Python code to solve problems.",
        tools=[HostedCodeInterpreterTool()],
    )
    
    # Test questions that require code execution
    questions = [
        "Use code to calculate the factorial of 100",
        "Calculate the first 15 numbers in the Fibonacci sequence",
        "Generate a list of prime numbers less than 100"
    ]
    
    for question in questions:
        print(f"User: {question}")
        result = await agent.run(question)
        print(f"Agent: {result.text}\n")
        
        # Extract and display the generated code if available
        if (
            isinstance(result.raw_representation, ChatResponse)
            and isinstance(result.raw_representation.raw_representation, OpenAIResponse)
            and len(result.raw_representation.raw_representation.output) > 0
            and isinstance(
                result.raw_representation.raw_representation.output[0], 
                ResponseCodeInterpreterToolCall
            )
        ):
            generated_code = result.raw_representation.raw_representation.output[0].code
            print(f"📝 Generated code:\n```python\n{generated_code}\n```\n")
        
        print("-" * 80 + "\n")

await code_interpreter_responses_demo()

=== Azure OpenAI Responses Client with Hosted Code Interpreter ===

User: Use code to calculate the factorial of 100


Agent: import math

# Calculate the factorial of 100
factorial_100 = math.factorial(100)
factorial_100 The factorial of 100 is:

93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000

📝 Generated code:
```python
import math

# Calculate the factorial of 100
factorial_100 = math.factorial(100)
factorial_100
```

--------------------------------------------------------------------------------

User: Calculate the first 15 numbers in the Fibonacci sequence
Agent: The Fibonacci sequence is a series of numbers where each number is the sum of the two preceding ones, usually starting with 0 and 1. I will calculate the first 15 numbers for you. def fibonacci_sequence(n):
    sequence = [0, 1]
    while len(sequence) < n:
        sequence.append(sequence[-1] + sequence[-2])
    return sequence

first_15_fibonacci = fibonacci_sequence(15)
first_15_fibonacci The first 15 numbers in the Fibonac

## 🧪 Case 4: Tool use examples with Azure AI Agent Client
---
examples demonstrating different ways to create and use Agents and tools with the Azure AI Agent Client from the agent_framework.

# Case 4: Enterprise Tools with Azure AI Agent Client
---
Microsoft Agent Framework provides powerful enterprise-grade hosted tools that integrate seamlessly with Azure services. These tools enable agents to perform advanced operations like web search, document retrieval, and external integrations.

## Hosted Tools Comparison

| Tool | Purpose | Service | Key Features |
|------|---------|---------|--------------|
| **HostedFileSearchTool** | Document/Data Search | Azure AI Search | ✅ Index-based search<br>✅ Semantic search<br>✅ Configurable ranking |
| **HostedWebSearchTool** | Real-time Web Search | Bing Grounding | ✅ Current information<br>✅ Source citations<br>✅ Web grounding |
| **Local MCP** | Custom Integrations | Local Server | ✅ Full control<br>✅ Custom logic<br>✅ Local resources |
| **Hosted MCP** | External Services | Remote Server | ✅ Third-party APIs<br>✅ Scalable<br>✅ Managed infrastructure |

### Prerequisites for Case 4:

**For Azure AI Search (Case 4.1):**
- Azure AI Search service with indexed data
- Search connection configured in Azure AI Project
- Index name (e.g., "hotels-sample-index")

**For Bing Grounding (Case 4.2):**
- Bing Grounding connection in Azure AI Project
- `BING_CONNECTION_NAME` or `BING_CONNECTION_ID` environment variable

**For MCP Tools (Case 4.3 & 4.4):**
- Understanding of Model Context Protocol (MCP)
- MCP server configuration (local or hosted)

## 🧪 Case 4.1: Azure AI Search with HostedFileSearchTool
---
Demonstrates using Azure AI Search to search through indexed hotel data and answer questions.

> **Note**: Before running this case, first execute the `1_basic-rag.ipynb` notebook in the `0_basic-rag` folder and create a connection to set up the required Azure AI Search index and vector store.

**Prerequisites:**
1. Set AZURE_AI_PROJECT_ENDPOINT and AZURE_AI_MODEL_DEPLOYMENT_NAME environment variables
2. Ensure you have an Azure AI Search connection configured in your Azure AI project
3. The search index "hotels-sample-index" should exist in your Azure AI Search service
   (you can create this using **1_basic-rag.ipynb** in the **0_basic-rag folder** with sample hotel data)

**Environment variables:**
- AZURE_AI_PROJECT_ENDPOINT: Your Azure AI project endpoint
- AZURE_AI_MODEL_DEPLOYMENT_NAME: The name of your model deployment

**Key Modules:**
- `HostedFileSearchTool` from `agent_framework`
- Uses Azure AI Search vector stores for semantic search capabilities

In [15]:
import os
from azure.identity.aio import AzureCliCredential
from agent_framework.azure import AzureAIAgentClient
from agent_framework import ChatAgent, HostedFileSearchTool, HostedVectorStoreContent
from azure.ai.agents.models import FileInfo, VectorStore

async def employee_search_demo():
    """
    Demonstrates Azure AI agent with file search capabilities using uploaded PDF.
    This example uploads an employee PDF file and creates a vector store for searching.
    """
    
    
    # Create Azure AI Agent client with credential
    credential = AzureCliCredential()
    
    file: FileInfo | None = None
    vector_store: VectorStore | None = None
    
    try:
        # Create a chat client and agent with Azure OpenAI
        azure_openai_chat_client = AzureOpenAIChatClient(
            deployment_name=azure_openai_deployment,
            endpoint=azure_openai_endpoint,
            api_key=azure_openai_key,
            api_version=azure_openai_version
        )

        azure_ai_agent_client = AzureAIAgentClient(
            async_credential=credential,
            project_endpoint=azure_project_endpoint,
            model_deployment_name=azure_ai_deployment
        )
            
        # 1. Upload employee PDF file
        pdf_file_path = "employees.pdf"
        print(f"Uploading file from: {pdf_file_path}")
        
        file = await azure_ai_agent_client.project_client.agents.files.upload_and_poll(
            file_path=str(pdf_file_path), 
            purpose="assistants"
        )
        print(f"Uploaded file, file ID: {file.id}")
        
        # 2. Create vector store with the uploaded file
        vector_store = await azure_ai_agent_client.project_client.agents.vector_stores.create_and_poll(
            file_ids=[file.id], 
            name="employee_vectorstore"
        )
        print(f"Created vector store, ID: {vector_store.id}\n")
        
        # 3. Create file search tool with the vector store
        file_search_tool = HostedFileSearchTool(
            inputs=[HostedVectorStoreContent(vector_store_id=vector_store.id)]
        )
        
        # 4. Create agent with file search capability
        agent = ChatAgent(
            #chat_client=azure_ai_agent_client,
            chat_client=azure_ai_agent_client,
            name="Employee Search Assistant",
            instructions="You are a helpful assistant that can search through uploaded employee files to answer questions about employees.",
            tools=file_search_tool
        )
        
        # 5. Test questions about employees
        USER_INPUTS = [
            "Who is the youngest employee?",
            "Who works in sales?",
            "I have a customer request, who can help me?"
        ]
        
        print("=== Azure AI Agent with File Search (Employee PDF) ===")
        print("This agent can search through employee data to answer questions.\n")
        
        # 6. Simulate conversation with the agent
        for user_input in USER_INPUTS:
            print(f"User: {user_input}")
            print("Agent: ", end="", flush=True)
            
            # Stream the response for better user experience
            async for chunk in agent.run_stream(user_input):
                if chunk.text:
                    print(chunk.text, end="", flush=True)
            print("\n" + "=" * 50 + "\n")
        
        print("Employee search conversation completed!")
            
    finally:
        # 7. Cleanup: Delete the vector store and file
        # Refresh client if needed (agent closes it when used as context manager)
        try:
            cleanup_client = AzureAIAgentClient(
                async_credential=AzureCliCredential(),
                project_endpoint=azure_project_endpoint,
                model_deployment_name=azure_ai_deployment
            )
            await cleanup_client.close()

            if azure_ai_agent_client:
                await azure_ai_agent_client.close()

            if vector_store:
                await cleanup_client.project_client.agents.vector_stores.delete(vector_store.id)
                print(f"Deleted vector store: {vector_store.id}")
            if file:
                await cleanup_client.project_client.agents.files.delete(file.id)
                print(f"Deleted file: {file.id}")
                
    
        except Exception as e:
            print(f"Cleanup warning: {e}")

await employee_search_demo()

Uploading file from: employees.pdf


Uploaded file, file ID: assistant-EqpJ4iYoGy7DUvDn8mF9Hv
Created vector store, ID: vs_5P43SpPT32ZmYzfJM8P0aXF8

=== Azure AI Agent with File Search (Employee PDF) ===
This agent can search through employee data to answer questions.

User: Who is the youngest employee?
Agent: The youngest employee is Alice Johnson from the Sales department, who is 24 years old【4:0†employees.pdf】.

User: Who works in sales?
Agent: The person who works in Sales is Alice Johnson. 

【4:0†employees.pdf】

User: I have a customer request, who can help me?
Agent: Could you please provide more details about the customer request? This will help me identify the right person who can assist you.

Employee search conversation completed!
Deleted vector store: vs_5P43SpPT32ZmYzfJM8P0aXF8
Deleted file: assistant-EqpJ4iYoGy7DUvDn8mF9Hv


In [16]:
import os
from agent_framework.azure import AzureAIAgentClient
from agent_framework import HostedFileSearchTool
from agent_framework import ChatAgent


async def ai_search_tool_responses_demo():

    try:
        # Create Azure AI Agent client with credential
        credential = AzureCliCredential()

        
        # Create file search tool with vector store
        azure_ai_search_tool = HostedFileSearchTool(
            additional_properties={
                "index_name": "hotels-sample-index",  # Name of your search index
                "query_type": "simple",  # Use simple search
                "top_k": 10,  # Get more comprehensive results
            },
        )

        azure_openai_chat_client = AzureOpenAIChatClient(
                deployment_name=azure_openai_deployment,
                endpoint=azure_openai_endpoint,
                api_key=azure_openai_key,
                api_version=azure_openai_version
            )

        # TODO: Not work with Azure AI Agent Client yet - https://github.com/microsoft/agent-framework/issues/1381
        # azure_ai_agent_client = AzureAIAgentClient(
        #     async_credential=credential,
        #     project_endpoint=project_endpoint,
        #     model_deployment_name=azure_ai_deployment
        # )
    
        # Create agent with file search capability
        agent = ChatAgent(
            chat_client=azure_openai_chat_client,
            name="Hotel Search Assistant",
            instructions="You are a helpful travel assistant that searches hotel information.",
            tools=azure_ai_search_tool
        )

        USER_INPUTS = [
            "Search hotels in USA and give me detailed information.",
        ]
        
        print("=== Azure AI Agent with Azure AI Search ===")
        print("This agent can search through hotel data to help you find accommodations.\n")

        # 3. Simulate conversation with the agent
        for user_input in USER_INPUTS:
            print(f"User: {user_input}")
            print("Agent: ", end="", flush=True)

            # Stream the response for better user experience
            async for chunk in agent.run_stream(user_input):
                if chunk.text:
                    print(chunk.text, end="", flush=True)
            print("\n" + "=" * 50 + "\n")

        print("Hotel search conversation completed!")

    except Exception as e:
        print(f"Error occurred: {e}")
    finally:
        cleanup_client = AzureAIAgentClient(
            async_credential=AzureCliCredential(),
            project_endpoint=azure_project_endpoint,
            model_deployment_name=azure_ai_deployment
        )
        await cleanup_client.close()

await ai_search_tool_responses_demo()

=== Azure AI Agent with Azure AI Search ===
This agent can search through hotel data to help you find accommodations.

User: Search hotels in USA and give me detailed information.
Agent: 

Sure! The

[2025-10-27 01:18:58 - /anaconda/envs/venv_agentlab/lib/python3.12/asyncio/base_events.py:1833 - ERROR] Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fe3b6454230>
[2025-10-27 01:18:58 - /anaconda/envs/venv_agentlab/lib/python3.12/asyncio/base_events.py:1833 - ERROR] Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x7fe3b4717bf0>, 3459979.628858652)])']
connector: <aiohttp.connector.TCPConnector object at 0x7fe3b63cd7f0>


 USA is a large country with many cities and a wide range of hotels. Could you please specify the city or region you are

[2025-10-27 01:18:58 - /anaconda/envs/venv_agentlab/lib/python3.12/asyncio/base_events.py:1833 - ERROR] Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fe3b4ce3d40>


 interested in? Also, let me know any preferences you have, such as hotel star rating, budget, or amenities. This will help me provide detailed and relevant hotel information for you.

Hotel search conversation completed!


## 🧪 Case 4.2: Bing Grounding with HostedWebSearchTool
---
Demonstrates using Bing Search API for web grounding to answer questions with real-time web data.

**Setup:**
- Azure AI Project with Bing Search connection
- Environment variables: `AZURE_AI_PROJECT_ENDPOINT`, `BING_CONNECTION_NAME`

**Key Modules:**
- `HostedWebSearchTool` from `agent_framework.azure_ai.tools`
- Provides grounded responses with web search results and citations

In [17]:
import os
from azure.identity.aio import AzureCliCredential
from agent_framework.azure import AzureAIAgentClient, AzureOpenAIChatClient
from agent_framework import HostedWebSearchTool
from agent_framework import ChatAgent


async def bing_grounding_demo():
    """
    Demonstrates using Bing Search API for web grounding to answer questions with real-time web data.
    """
    
    try:
        # Get Azure AI Project endpoint
        bing_connection_id = os.environ.get("BING_GROUNDING_CONNECTION_ID")
        
        # Create Azure AI Agent client with credential
        credential = AzureCliCredential()
        
        # Create web search tool with Bing connection
        web_search_tool = HostedWebSearchTool(
            
            additional_properties={
                "connection_id": bing_connection_id,
                "top_k": 3,  # Get top 3 search results
            }
        )
        
        # TODO: Not work with Azure OpenAI Chat Client
        # azure_openai_chat_client = AzureOpenAIChatClient(
        #     deployment_name=azure_openai_deployment,
        #     endpoint=azure_openai_endpoint,
        #     api_key=azure_openai_key,
        #     api_version=azure_openai_version
        # )
        
        azure_ai_agent_client = AzureAIAgentClient(
            async_credential=credential,
            project_endpoint=azure_project_endpoint,
            model_deployment_name=azure_ai_deployment
        )
        
        # Create agent with web search capability
        agent = ChatAgent(
            #chat_client=azure_openai_chat_client,
            chat_client=azure_ai_agent_client,
            name="Web Research Assistant",
            instructions="You are a helpful research assistant. Use web search to find current information and provide grounded answers with citations.",
            tools=[web_search_tool]
        )
        
        # Test questions that require web search
        USER_INPUTS = [
            "Tell me what day it is today and What is the latest exciting news in South Korea?"
        ]
        
        print("=== Azure AI Agent with Bing Grounding Search ===")
        print("This agent can search the web for current information and provide grounded answers.\n")
        
        # Simulate conversation with the agent
        for user_input in USER_INPUTS:
            print(f"User: {user_input}")
            print("Agent: ", end="", flush=True)
            
            # Stream the response for better user experience
            async for chunk in agent.run_stream(user_input):
                if chunk.text:
                    print(chunk.text, end="", flush=True)
            print("\n" + "=" * 50 + "\n")
        
        print("Web search conversation completed!")
    
    except Exception as e:
        print(f"Error occurred: {e}")
    finally:
        # Cleanup: Close Azure AI Agent client if it was created
        try:
            await azure_ai_agent_client.close()    
        except Exception as cleanup_error:
            print(f"Cleanup warning: {cleanup_error}")

await bing_grounding_demo()

=== Azure AI Agent with Bing Grounding Search ===
This agent can search the web for current information and provide grounded answers.

User: Tell me what day it is today and What is the latest exciting news in South Korea?
Agent: I'm sorry, but I cannot assist with that request.

Web search conversation completed!


## 🧪 Case 4.3: Local MCP Server Integration
---
Demonstrates connecting to a local Model Context Protocol (MCP) server to extend agent capabilities with custom tools.

**Setup:**
- Local MCP server running (e.g., file system tools, database connectors)
- MCP server URL or configuration
- Environment variable: `AZURE_OPENAI_ENDPOINT`

**Key Modules:**
- Discovers and registers tools from MCP server dynamically
- Works with any MCP-compliant server implementation

In [18]:
import os
from azure.identity.aio import AzureCliCredential
from agent_framework.azure import AzureOpenAIChatClient, AzureAIAgentClient
from agent_framework import ChatAgent, MCPStreamableHTTPTool


async def local_mcp_demo():
    """
    Demonstrates connecting to a local Model Context Protocol (MCP) server 
    to extend agent capabilities with custom tools.
    
    This example shows how to use Microsoft Learn MCP server with Azure AI Agent.
    """
    
    try:
        # Create Azure AI Agent client with credential
        credential = AzureCliCredential()
        
        print("=== Azure AI Agent with Local MCP Server ===")
        print("This agent can use MCP tools to access Microsoft documentation.\n")
        
        # Connect to local MCP server and create agent with MCP tools
        async with (
            AzureAIAgentClient(
                async_credential=credential,
                project_endpoint=azure_project_endpoint,
                model_deployment_name=azure_ai_deployment
            ) as azure_ai_agent_client,
            MCPStreamableHTTPTool(
                name="Local Microsoft Learn MCP",
                url="https://learn.microsoft.com/api/mcp",
            ) as mcp_server
        ):
            # Create agent with MCP tools
            agent = ChatAgent(
                chat_client=azure_ai_agent_client,
                name="Local MCP Assistant",
                instructions="You are a helpful assistant that can help with Microsoft documentation questions using MCP tools.",
                tools=mcp_server,
                store=True
            )
            
            # Test questions that use MCP tools
            USER_INPUTS = [
                "How to create an Azure storage account using az cli? response with a simple answer and an example.",
                "What is Microsoft Agent Framework? response with key concepts and a main python code example."
            ]
            
            # Simulate conversation with the agent
            for user_input in USER_INPUTS:
                print(f"User: {user_input}")
                print("Agent: ", end="", flush=True)
                
                # Stream the response for better user experience
                async for chunk in agent.run_stream(user_input):
                    if chunk.text:
                        print(chunk.text, end="", flush=True)
                print("\n" + "=" * 50 + "\n")
            
            print("MCP tool conversation completed!")
    
    except Exception as e:
        print(f"Error occurred: {e}")
        import traceback
        traceback.print_exc()

await local_mcp_demo()

=== Azure AI Agent with Local MCP Server ===
This agent can use MCP tools to access Microsoft documentation.



User: How to create an Azure storage account using az cli? response with a simple answer and an example.
Agent: To create an Azure storage account using Azure CLI, use the `az storage account create` command with parameters for resource group, storage account name, and location.

Example:
```bash
az storage account create --name mystorageaccount --resource-group myResourceGroup --location eastus --sku Standard_LRS
```

User: What is Microsoft Agent Framework? response with key concepts and a main python code example.
Agent: Microsoft Agent Framework is a comprehensive framework provided by Microsoft to build and manage intelligent AI agents equipped with advanced conversational AI capabilities. It supports multiple types of agents, all derived from a common base class called AIAgent, enabling a consistent interface and allowing for building complex functionalities like multi-agent orchestrations.

Key concepts of Microsoft Agent Framework include:
1. **Simple Agents Based on Inference 

## 🧪 Case 4.4: Hosted MCP Server with Human in the loop (Azure AI Project)
---
Demonstrates using MCP servers hosted in Azure AI Project with human-in-the-loop capabilities

**Setup:**
- Azure AI Project with hosted MCP server connection
- MCP server deployed and registered in Azure AI Project
- Environment variable: `AZURE_AI_PROJECT_ENDPOINT`

**Key Modules:**
- `HostedMCPTool` from `agent_framework`
- Connects to MCP servers hosted in Azure AI infrastructure
- Human-in-the-loop support for review and approval workflows
- Provides secure, managed access to MCP tools without local server setup

In [19]:
import os
from azure.identity.aio import AzureCliCredential
from agent_framework.azure import AzureAIAgentClient
from agent_framework import HostedMCPTool, ChatMessage
from agent_framework import ChatAgent
from typing import Any


async def handle_approvals_with_thread(query: str, agent, thread):
    """Handle user approval requests for function calls."""
    result = await agent.run(query, thread=thread, store=True)
    
    while len(result.user_input_requests) > 0:
        new_input: list[Any] = []
        for user_input_needed in result.user_input_requests:
            print(
                f"\n⚠️ Function call approval needed: {user_input_needed.function_call.name}"
                f"\n   Arguments: {user_input_needed.function_call.arguments}"
            )
            # Visual feedback with dots for 1 second
            for _ in range(5):
                print(".", end="", flush=True)
                await asyncio.sleep(0.33)
            print("user approval simulated. yes! \n")
            # Auto-approve for demo purposes (in production, you'd want user input)
            user_approval = "y"  # or use: input("Approve? (y/n): ")
            new_input.append(
                ChatMessage(
                    role="user",
                    contents=[user_input_needed.create_response(user_approval.lower() == "y")],
                )
            )
        result = await agent.run(new_input, thread=thread, store=True)
    
    return result


async def hosted_mcp_demo():
    """
    Demonstrates using MCP servers hosted in Azure AI Project for enterprise-ready tool integration.
    
    This example shows how to connect to a hosted MCP server with proper approval workflow.
    Based on: https://github.com/microsoft/agent-framework/tree/main/python/samples
    """
    
    try:

        # Create Azure AI Agent client with credential
        credential = AzureCliCredential()
        
        print("=== Azure AI Agent with Hosted MCP Server ===")
        print("This agent uses hosted MCP tools to access Microsoft documentation.\n")
        
        async with AzureAIAgentClient(
            async_credential=credential,
            project_endpoint=azure_project_endpoint,
            model_deployment_name=azure_ai_deployment,
            ) as chat_client:
            
            # Enable Azure AI observability (optional but recommended)
            #await chat_client.setup_azure_ai_observability()
            
            # Create agent with hosted MCP tools using create_agent method
            agent = chat_client.create_agent(
                name="Hosted MCP Assistant",
                instructions="You are a helpful assistant that can help with Microsoft documentation questions using MCP tools.",
                tools=HostedMCPTool(
                    name="Microsoft Learn MCP",
                    url="https://learn.microsoft.com/api/mcp",
                )
            )
            
            # Create a new thread for conversation
            thread = agent.get_new_thread()
            
            # Test questions using MCP tools
            USER_INPUTS = [
                "How to create an Azure storage account using az cli?",
                "What is Microsoft Agent Framework?"
            ]
            
            # Simulate conversation with approval workflow
            for user_input in USER_INPUTS:
                print(f"User: {user_input}")
                print("Agent: ", end="", flush=True)
                
                # Handle query with approval workflow
                result = await handle_approvals_with_thread(user_input, agent, thread)
                print(f"{result.text}")
                print("\n" + "=" * 50 + "\n")
            
            print("Hosted MCP tool conversation completed!")
    
    except Exception as e:
        print(f"Error occurred: {e}")
        import traceback

    finally:
        # Cleanup: Close Azure AI Agent client if it was created
        try:
            await chat_client.close()    
        except Exception as cleanup_error:
            print(f"Cleanup warning: {cleanup_error}")

await hosted_mcp_demo()

=== Azure AI Agent with Hosted MCP Server ===
This agent uses hosted MCP tools to access Microsoft documentation.

User: How to create an Azure storage account using az cli?
Agent: 
⚠️ Function call approval needed: microsoft_docs_search
   Arguments: {"query":"create Azure storage account using az cli","question":"How to create an Azure storage account using az cli?"}
.....user approval simulated. yes! 

To create an Azure storage account using Azure CLI, you can use the `az storage account create` command. Here's a basic example:

```azurecli
az storage account create \
  --name <account-name> \
  --resource-group <resource-group-name> \
  --location <location> \
  --sku Standard_RAGRS \
  --kind StorageV2 \
  --min-tls-version TLS1_2 \
  --allow-blob-public-access false
```

Replace the placeholders with your own values:
- `<account-name>`: The unique name for your storage account.
- `<resource-group-name>`: The name of your Azure resource group.
- `<location>`: The Azure region whe

## 📊 Case 4 Comparison: Enterprise Tools

| Feature | AI Search (4.1) | Bing Grounding (4.2) | Local MCP (4.3) | Hosted MCP (4.4) |
|---------|----------------|---------------------|----------------|------------------|
| **Tool Type** | `HostedFileSearchTool` | `HostedWebSearchTool` | `MCPClient` | `HostedMCPTool` |
| **Client Type** | `AzureAIAgentClient` | `AzureAIAgentClient` | `AzureOpenAIChatClient` | `AzureAIAgentClient` |
| **Data Source** | Azure AI Search vector stores | Bing Search API | Local MCP server process | Azure-hosted MCP server |
| **Setup Complexity** | Medium (requires vector store) | Low (requires Bing connection) | Medium (requires local server) | Low (requires Azure connection) |
| **Use Cases** | Internal document search, knowledge base | Real-time web information | Custom local tools, file system | Enterprise-grade custom tools |
| **Authentication** | `DefaultAzureCredential` | `DefaultAzureCredential` | `AzureCliCredential` | `DefaultAzureCredential` |
| **Grounding** | Citations from indexed docs | Web citations and sources | N/A (custom tool output) | Depends on MCP server |
| **Scalability** | High (Azure managed) | High (Azure managed) | Low (local process) | High (Azure managed) |
| **Customization** | Limited to indexed data | Limited to web search | Full (custom MCP tools) | Full (custom MCP tools) |
| **Cost** | AI Search + AI Project | Bing API + AI Project | Azure OpenAI only | MCP hosting + AI Project |

**Key Takeaways:**
- **AI Search (4.1)**: Best for searching through your own documents and knowledge bases with semantic search
- **Bing Grounding (4.2)**: Best for real-time web information with source citations
- **Local MCP (4.3)**: Best for development and testing with custom tools on local machine
- **Hosted MCP (4.4)**: Best for production deployment of custom tools with enterprise security and scalability